In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

In [4]:
import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import roc_auc_score

In [5]:
# 数据集：
path="/Users/sun/CLionProjects/LightGBM/examples/regression/"
df_train=pd.read_csv(path+"regression.train",header=None,sep='\t')
df_test=pd.read_csv(path+"regression.test",header=None,sep='\t')

In [8]:
# 矩阵：
y_train = df_train[0].values
y_test = df_test[0].values
X_train = df_train.drop(0, axis=1).values
X_test = df_test.drop(0, axis=1).values

In [9]:
# 转为LightGBM数据集：
lgb_train = lgb.Dataset(X_train, y_train)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [45]:
# 训练：
# {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 150,
    'max_depth': 7,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'max_bin':200
}
gbm = lgb.train(
    params,
    lgb_train,
#     num_boost_round=20,
#     valid_sets=lgb_eval,
#     early_stopping_rounds=5
)

In [46]:
# 模型保存：
gbm.save_model('model.txt')

In [47]:
# 预测：
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [48]:
# 评分：
roc_auc_score(y_test, y_pred)

0.8330430856553148

In [ ]:
# 模型存为JSON：
model_json = gbm.dump_model()
with open('model.json', 'w+') as f:
    json.dump(model_json, f, indent=4)
print('Feature names:', gbm.feature_name())
print('Calculate feature importances...')
# feature importances
print('Feature importances:', list(gbm.feature_importance()))